In [3]:
import sys
sys.path.append('../..')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns

from sentiment import sentiment_vader
from imdb_api.imdbscraper import ImdbScraper

from scipy import stats

### Trailers

In [ ]:
file_list = ['hbo', 'amazon', 'netflix', 'disney']
dfs = []
for file in file_list:
    df = pd.read_csv('../data/' + file + '.csv')
    df['network'] = file
    dfs.append(df)

cols = ['channelId', 'network', 'videoId', 'videoTitle', 'publishTime']
trailers = pd.concat(dfs)[cols]

### Comments

In [ ]:
file_list = ['hbo_comments', 'amazon_comments', 'netflix_comments', 'disney_comments']
dfs = []
for file in file_list:
    df = pd.read_csv('../../comments/' + file + '.csv')
    dfs.append(df)

comments = pd.concat(dfs)
cols = ['videoId', 'commentId', 'textOriginal', 'likeCount', 'publishedAt']
comments = comments[cols]

In [ ]:
comments.to_csv('comments.csv', index=False)

### Mapping imdb and youtube

In [ ]:
file_list = ['hbo', 'amazon', 'netflix', 'disney']
dfs = []
for file in file_list:
    df = pd.read_csv('../../data/match/' + file + '_match.csv', delimiter=';')
    dfs.append(df)

match = pd.concat(dfs).dropna()

### IMDb

In [ ]:
imdb = pd.read_csv('../imdb/title.basics.tsv', delimiter='\t')
ratings = pd.read_csv('../imdb/title.ratings.tsv', delimiter='\t')
imdb = imdb.merge(ratings, on='tconst')

### IMDb release dates

In [ ]:
# scraper = ImdbScraper()
# scraper.scrape_dates_alternate(match.tconst, verbose=True, timeout=1)

In [ ]:
# release_dates = pd.DataFrame(scraper.data)
# release_dates['releaseDateUS'] = pd.to_datetime(release_dates.release_date_us)

In [ ]:
# release_dates.to_csv('release_dates.csv', index=False)

In [ ]:
release_dates = pd.read_csv('../data/release_dates/release_dates.csv')
cols = ['id', 'release_date_us']
release_dates = release_dates[cols]
release_dates = release_dates.rename(columns={'id':'tconst', 'release_date_us': 'releaseDateUS'})

### Annotations

In [ ]:
annot = pd.read_csv('../comments/annotated.csv')
annot = annot[['commentId', 'sentiment', 'annotator']]

In [ ]:
annot_agg = annot.groupby('commentId').agg(
    sentimentLabel=('sentiment', stats.mode)
).reset_index()

annot_agg.sentimentLabel = annot_agg.sentimentLabel.str[0].str[0]

sentiment_map = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
annot_agg['sentimentScore'] = annot_agg.sentimentLabel.map(sentiment_map)

### ReturnYoutTubeDislikes

In [ ]:
ryd = pd.read_csv('../data/Likes_Dislikes_Viewcount.csv')
ryd = ryd[['videoId', 'likes', 'dislikes', 'viewCount']]

### The big join

In [ ]:
df = trailers.merge(comments, on='videoId')
df = df.merge(ryd, on='videoId', how='left')
df = df.merge(match, on='videoId')
df = df.merge(imdb, on='tconst')
df = df.merge(release_dates, on='tconst', how='left')
df = df.merge(annot_agg, on='commentId', how='left')

### Calculate comment date offset

In [ ]:
dt = pd.to_datetime(df.publishedAt)
comment_date = dt.dt.date

In [ ]:
df['releaseDateUS'] = pd.to_datetime(df.releaseDateUS)

In [ ]:
df['commentDateOffset'] = (pd.to_datetime(comment_date) - df.releaseDateUS)
df['commentDateOffset'] = df.commentDateOffset.astype('timedelta64[D]').astype('float')

In [ ]:
df.to_csv('dataset_no_sentiment.csv', index=False)

### Add sentiment

In [ ]:
df['sentimentPredictedRaw'] = df.textOriginal.astype(str).apply(sentiment_vader)

In [ ]:
sentiment_map = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
df['sentimentPredictedScore'] = df.sentimentPredictedRaw.str[-1].map(sentiment_map)

In [ ]:
df.to_csv('final_data.csv', index=False)